# Machine Translation using RNN GRU
This use-case provides a demo of how RNN and GRU can be used for machine translation of languages using seq2seq.
In our use-case, we will use spa.txt as training set which has a translation of English words to Spanish words. The database can be downloaded from the URL https://www.dropbox.com/s/poqpk4528x4nkp3/spa.txt


*Workflow:*

1. Understanding the problem
2. Collecting the data
3. Data preprocessing 
4. Build seq2seq Model using Grated Recurrent Unit (GRU)
5. Training the dataset
6. Translate



## 1. Understanding the problem 
The problem statement is to translate the English text into Spanish. For this purpose, we will be using RNN-GRU model using sequence modeling. 

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


##Download data and put it in pandas DataFrame

In [0]:
!wget https://www.dropbox.com/s/poqpk4528x4nkp3/spa.txt #Download the datafile from this site

--2020-05-19 11:35:31--  https://www.dropbox.com/s/poqpk4528x4nkp3/spa.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/poqpk4528x4nkp3/spa.txt [following]
--2020-05-19 11:35:32--  https://www.dropbox.com/s/raw/poqpk4528x4nkp3/spa.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc27a479642c7e7927f96736814b.dl.dropboxusercontent.com/cd/0/inline/A4DnSRCMtFXkvHahaqhayXtTmp2ypr0cydzLs5apavKKuqojfjflK1fRviUD-TH9BDYT5rGLycIaUGpvxnVKqx3O1pFATr0PNFFQ_MXljGDXe0tKdGPHfCL4V3IIhEq4GYo/file# [following]
--2020-05-19 11:35:32--  https://uc27a479642c7e7927f96736814b.dl.dropboxusercontent.com/cd/0/inline/A4DnSRCMtFXkvHahaqhayXtTmp2ypr0cydzLs5apavKKuqojfjflK1fRviUD-TH9BDYT5rGLycIaUGpvxnVKqx3O1pFATr0PNFFQ_MXljGDXe0tKdGPHfCL4V3IIhE

In [0]:
fi = open("/content/spa.txt",encoding='utf-8',mode='r')
data = fi.read()

In [0]:
df=[]
for line in data.split('\n'):
  df.append(line.split('\t'))

In [0]:
lines=pd.DataFrame(df)
lines.head(20)
lines = lines.iloc[:,:2]
lines.columns=['english','spanish']

In [0]:
lines.dropna(inplace=True)
lines.shape

(123770, 2)

In [0]:
lines.head()

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,VÃ¡yase.
4,Hi.,Hola.


In [0]:
#lines=lines[~pd.isnull(lines['english'])]

In [0]:
lines.drop_duplicates(inplace=True)
lines=lines.sample(n=120000,random_state=42)
lines.shape

(120000, 2)

##Clean the text data

In [0]:
# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['spanish']=lines['spanish'].apply(lambda x: x.lower())

# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['spanish']=lines['spanish'].apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['spanish']=lines['spanish'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['spanish']=lines['spanish'].apply(lambda x: x.translate(remove_digits))


# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['spanish']=lines['spanish'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['spanish']=lines['spanish'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines['spanish'] = lines['spanish'].apply(lambda x : 'START_ '+ x + ' _END')

In [0]:
lines.head()

,english,spanish
112706,tom entered his office and saw a box on his desk,START_ tom entrã³ a su oficina y vio una caja sobre el escritorio _END
39299,the trees were in a row,START_ los ã¡rboles estaban en lã­nea _END
115199,my uncle went to mexico in,START_ never to come back _END
39449,this book is very heavy,START_ este libro es muy pesado _END
11509,im sure of this,START_ estoy bastante seguro _END


In [0]:
### Get English and Spanish Vocabulary
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_spanish_words=set()
for spa in lines['spanish']:
    for word in spa.split():
        if word not in all_spanish_words:
            all_spanish_words.add(word)

lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_spa_sentence']=lines['spanish'].apply(lambda x:len(x.split(" ")))
lines.head()

,english,spanish,length_eng_sentence,length_spa_sentence
112706,tom entered his office and saw a box on his desk,START_ tom entrã³ a su oficina y vio una caja sobre el escritorio _END,11,14
39299,the trees were in a row,START_ los ã¡rboles estaban en lã­nea _END,6,7
115199,my uncle went to mexico in,START_ never to come back _END,6,6
39449,this book is very heavy,START_ este libro es muy pesado _END,5,7
11509,im sure of this,START_ estoy bastante seguro _END,4,5


In [0]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_spa_sentence']<=20]
print("maximum length of Spanish Sentence ",max(lines['length_spa_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

max_length_src=max(lines['length_spa_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_spanish_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_spanish_words)
num_encoder_tokens, num_decoder_tokens
num_decoder_tokens += 1 #for zero padding

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)
lines.head(10)

maximum length of Spanish Sentence  20
maximum length of English Sentence  20


,english,spanish,length_eng_sentence,length_spa_sentence
80235,lets do only one thing at a time,START_ hagamos las cosas de una en una _END,8,9
8569,i read the book,START_ leo el libro _END,4,5
118126,a girl gave me an orange in exchange for a piece of cake,START_ una chica me dio una naranja a cambio de un trozo de tarta _END,13,15
103160,i should have been watching more carefully,START_ deberã­a haber estado observando con mã¡s atenciã³n _END,7,9
8590,i see his house,START_ yo veo su casa _END,4,6
66193,whats your area of expertise,START_ â¿en quã© campo eres experto _END,5,7
20251,do you want my help,START_ â¿quieres mi ayuda _END,5,5
104939,i think its time for me to go back to work,START_ creo que es hora de que regrese al trabajo _END,11,11
22941,tom is kind to mary,START_ tom es gentil con mary _END,5,7
7488,wait out front,START_ espera enfrente _END,3,4


In [0]:
X, y = lines['english'], lines['spanish']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((95880,), (23970,))

##Generator to generate the data for training

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

##Create the model

In [0]:
#Model

latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None, ))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.


decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    3942300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    8592000     input_2[0][0]                    
____________________________________________________________________________________________

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

##Train the model

In [0]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
749/749 [==============================] - 258s 344ms/step - loss: 1.8005 - val_loss: 1.6503
Epoch 2/10
749/749 [==============================] - 249s 332ms/step - loss: 1.3871 - val_loss: 1.3706
Epoch 3/10
749/749 [==============================] - 248s 331ms/step - loss: 1.1880 - val_loss: 1.1528
Epoch 4/10
749/749 [==============================] - 246s 328ms/step - loss: 1.0544 - val_loss: 1.0443
Epoch 5/10
749/749 [==============================] - 245s 327ms/step - loss: 0.9563 - val_loss: 1.0547
Epoch 6/10
749/749 [==============================] - 246s 329ms/step - loss: 0.8843 - val_loss: 1.0580
Epoch 7/10
749/749 [==============================] - 251s 335ms/step - loss: 0.8254 - val_loss: 1.0533
Epoch 8/10
749/749 [==============================] - 254s 339ms/step - loss: 0.7771 - val_loss: 0.9835
Epoch 9/10
749/749 [==============================] - 255s 340ms/step - loss: 0.7357 - val_loss: 1.0174
Epoch 10/10
749/749 [==============================] - 254s 339m

In [0]:
model.save_weights('nmt_weights.h5')

##Generate Translation

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [0]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Spanish Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Spanish Translation:', decoded_sentence[:-4])

Input English sentence: should i open the windows
Actual Hindi Translation:  â¿deberã­a abrir las ventanas 
Predicted Hindi Translation:  â¿deberã­a abrir las ventanas 
